In [0]:
import numpy as np
import random
import matplotlib.pyplot as plt
from abc import ABC, abstractmethod


In [0]:
np.random.seed()
random.seed()

In [0]:
class Tools:
    def __init__(self, name, price, category, available):
        self.name = name
        self.price = price
        self.category = category
        self.available = available 
        
    def __repr__(self):
        return (f"{self.__class__.__name__}"
                f"({self.name}, ${self.price}, "
                f"{self.category}, {self.available})")

In [0]:
np.random.seed()
random.seed()

class Tools_set:
    def __init__(self, num_tools=20):
        self.num_tools = num_tools
        self.tool_categories = ["Painting", "Concrete", "Plumbing", "Woodwork", "Yardwork"]
        self.tool_prices = [1, 2, 3, 4, 5]
        self.num_tool_categories = len(self.tool_categories)
        
        self.available_tools = []
        self.rented_tools = []
        
        self.gtools()
        
    def gtools(self):
        for tool_i in range(self.num_tools):
            random_idx = np.random.randint(0, self.num_tool_categories)
            name = f"T{tool_i+1:02d}-{self.tool_categories[random_idx]}"
            price = self.tool_prices[random_idx]
            category = self.tool_categories[random_idx]
            available = True
            tool_obj = Tool(name, price, category, available)
            self.available_tools.append(tool_obj)
    
    def __len__(self):
        return len(self.available_tools)
        
    def rent(self, chosen_tools): 
        for chosen_tool in chosen_tools:
            for tool_i, tool in enumerate(self.available_tools):
                if tool.name == chosen_tool.name:
                    tool.available = False
                    self.rented_tools.append(chosen_tool)
                    del self.available_tools[tool_i]
                    break
    
    def restock(self, tools):
        for tool in tools:
            tool.available = True
            self.available_tools.append(tool)
    
    def print_tool_list(self):
        for t in self.available_tools:
            print(t)
            
    def __call__(self):
        return self.available_tools  
    
    def __contains__(self, other):
        pass
    
    def __iter__(self):
        for tool in self.available_tools:
            yield tool

In [0]:
class Customer:
    def __init__(self, name, customer_type, num_tools_wanted, num_nights, num_tools_rented):
        self.name = name
        self.customer_type = customer_type
        self.num_tools_wanted = num_tools_wanted
        self.num_nights = num_nights
        self.num_tools_rented = num_tools_rented
        
    def __repr__(self):
        return (f"{self.__class__.__name__}"
                f"({self.name}, {self.customer_type}, "
                f"{self.num_tools_wanted}, {self.num_nights}), NUM_TOOLS_RENTED={self.num_tools_rented}")

In [0]:
np.random.seed()
random.seed()

class CustomerCollection:
    def __init__(self, num_customers=10, num_customer_types=3):
        self.num_customers = num_customers
        self.num_customer_types = num_customer_types
        self.customer_types = ["Casual", "Business", "Regular"]
        self.num_tools = [[1,2], [3], [1,2,3]]
        self.num_nights = [[1,2], [7], [3,4,5]]

        self.customer_objects = []
        
        self.generate_customers()
        
    def generate_customers(self):
        for customer_i in range(self.num_customers):
            random_idx = np.random.randint(0, self.num_customer_types)
 
            customer_params = {
                "name"             : f"C{customer_i+1:02d}-{self.customer_types[random_idx][0]}", 
                "customer_type"    : self.customer_types[random_idx], 
                "num_tools_wanted" : random.sample(self.num_tools[random_idx], 1)[0], 
                "num_nights"       : random.sample(self.num_nights[random_idx], 1)[0], 
                "num_tools_rented" : 0
            }

            customer_obj = Customer(**customer_params)
            self.customer_objects.append(customer_obj)
                
    def get_customers(self):
        customer_pool = [customer for customer in self.customer_objects if customer.num_tools_rented < 3]  # Only customers with 0, 1, 2 active tools
        todays_customers = random.sample(customer_pool, np.random.randint(len(customer_pool)))        
        return todays_customers
            
    def return_tool(self, customer_name, num_tools_returned):
        for customer in self.customer_objects:
            if customer.name == customer_name:
                customer.num_tools_rented -= num_tools_returned
                break
                
    def print_customer_list(self):
        for c in self.customer_objects:
            print(c)
            
    def __call__(self):
        return self.customer_objects
    
    def __contains__(self, other):
        pass
    
    def __iter__(self):
        for customer in self.customer_objects:
            yield customer

In [0]:
class Rental:
    def __init__(self, customer_name, tools_rented, 
                 num_rent_nights, total_price,
                 day_rented, day_due, returned):
        
        self.customer_name = customer_name
        self.tools_rented = tools_rented
        self.num_rent_nights = num_rent_nights
        self.total_price = total_price
        self.day_rented = day_rented
        self.day_due = day_due
        self.returned = returned
        
    def set_returned(self):
        self.returned = True
        
    def __repr__(self):
        return (f"{self.__class__.__name__}, Customer: {self.customer_name}, "
                f"({self.tools_rented}, RentDate: {self.day_rented+1}, "
                f"DueDate: {self.day_due+1}, TotalPrice: ${self.total_price}, RETURNED: {self.returned})")

In [0]:
class Collection:
    def __init__(self):
        self.rental_objects = []
        
    def create_rental(self, customer, chosen_tools, total_price, day_i):
        # For a given customer, make rental record
        rental_params = {
                "customer_name"   : customer.name, 
                "tools_rented"    : chosen_tools, 
                "num_rent_nights" : customer.num_nights, 
                "total_price"     : total_price,
                "day_rented"      : day_i, 
                "day_due"         : day_i + customer.num_nights, 
                "returned"        : False
            }
            
        rental = Rental(**rental_params)
        return rental
    
    def append_rental(self, rental):
        self.rental_objects.append(rental)
            
    def __call__(self):
        return self.rental_objects
    
    def __contains__(self, other):
        pass
    
    def __iter__(self):
        for rental in self.rental_objects:
            yield rental

In [0]:
class Storefa(ABC):
    def __init__(self, tools, rental_collection, customers):
        self.tools = ToolCollection()               
        self.rental_collection = RentalCollection() 
        self.customers = CustomerCollection()
        
    def get_num_tools_available(self):
        pass
            
    def get_total_money(self):
        pass
    
    def update_money(self): 
        pass
        
    def update_customers(self): 
        pass
        
    def process_returns(self): 
        pass
    
    def rent_tools(self): 
        pass
    
    def retrieve_rental_records(self):
        pass

In [0]:
class Storef(StoreFacadeAbstract):
    def __init__(self):
        self.tools = ToolCollection()               # Inventory
        self.rental_collection = RentalCollection() # Rentals
        self.customers = CustomerCollection()
        
        self.total_money = 0
        self.num_avail = 20
        
        self.active_rentals = []
        self.completed_rentals = []
    
    def get_num_tools_available(self):
        self.num_avail = len(self.tools)
        return self.num_avail
    
    def get_available_tools(self):
        return self.tools()
            
    def get_total_money(self):
        return self.total_money
    
    def update_money(self, today_money): 
        self.total_money += today_money
        
    def update_customers(self, customer): 
        customer.num_tools_rented += customer.num_tools_wanted
        
    def process_returns(self, day_i): 
        for rental_receipt in self.rental_collection.rental_objects:
            if rental_receipt.day_due == day_i:
                rental_receipt.set_returned()
                self.tools.restock(rental_receipt.tools_rented)
                self.customers.return_tool(rental_receipt.customer_name, len(rental_receipt.tools_rented))                
                
    def rent_tools(self, day_i): 
        todays_customers = self.customers.get_customers()
        
        todays_customer_names = []
        for customer_i, customer in enumerate(todays_customers):
            if customer.num_tools_wanted <= self.num_avail and customer.num_tools_wanted + customer.num_tools_rented <= 3:

                available_tools = self.get_available_tools()
                chosen_tools = random.sample(available_tools, customer.num_tools_wanted)
                self.tools.rent(chosen_tools)
                total_price = sum([tool.price for tool in chosen_tools] * customer.num_nights)

                # Make a Rental record for this customer & update rental records
                rental = self.rental_collection.create_rental(customer, chosen_tools, total_price, day_i)
                self.rental_collection.append_rental(rental)
                                
                # Update total earning
                self.update_money(rental.total_price)

                # Each day, update num_tools_rented for Customer objects that rent
                self.update_customers(customer)

                # Update number of tools in inventory
                self.num_avail = self.get_num_tools_available()
                
    def retrieve_rental_records(self):
        for rental in self.rental_collection():
            if rental.returned:
                self.completed_rentals.append(rental)
            else:
                self.active_rentals.append(rental)
            
        rental_history = {"Active rentals": self.active_rentals,
                          "Completed rentals": self.completed_rentals}
        return rental_history

In [0]:
class Sum_Report:
    def __init__(self, tools, total_money, rental_history):
        self.tools = tools
        self.total_money = total_money
        self.rental_history = rental_history

    def format_rental(self, rental_type):
        report = ""
        for rental_i, rental in enumerate(rental_type):
            customer_name = rental.customer_name
            tools_rented = ", ".join([tool.name for tool in rental.tools_rented])
            rental_length = rental.num_rent_nights
            rental_cost = rental.total_price
            rental_report_1 = f"\t{rental_i+1}. Customer: {customer_name}" 
            rental_report_2 = rental_report_1 + f"\n\tTools rented: {tools_rented}"
            rental_report_3 = rental_report_2 + f"\n\tRental length: {rental_length} days" 
            rental_report_4 = rental_report_3 + f"\n\tCost: ${rental_cost}\n\n"
            report += rental_report_4
        return report

    def generate_report(self):
        tool_names = ", ".join([tool.name for tool in self.tools])
        num_tools = len(self.tools)
        
        active_rentals = self.format_rental(self.rental_history["Active rentals"])
        completed_rentals = self.format_rental(self.rental_history["Completed rentals"])

        print(f"---------------- REPORT ----------------")
        print(f"Total income: ${self.total_money} \n")
        print(f"Current number of tools in store: {num_tools} \n")
        print(f"Names of tools in store: {tool_names} \n")
        print(f"Rentals which are active: ")
        print(f"{active_rentals}")
        print(f"Rentals whcih completed: ")
        print(f"{completed_rentals}")

In [0]:
np.random.seed()
random.seed()

def main():
    num_days = 40
    print_simulation = False

    store = StoreFacade()

    for day_n in range(num_days):

        store.process_returns(day_n)

        num_tools_available_in_store = store.get_num_tools_available()

        if num_tools_available_in_store != 0:
            store.rent_tools(day_n)   

        if print_simulation:
            print(f"---------------- DAY {day_n+1} ----------------")
            print(f"Number of available tools = {num_tools_available_in_store} \n")
            print(f"Today's available tools: {store.get_available_tools()}\n")

    money_earned = store.get_total_money()
    tools_in_store = store.get_available_tools()
    rental_history = store.retrieve_rental_records()

    report = Sum_Report(tools_in_store, money_earned, rental_history)
    report.generate_report()

if __name__ == "__main__":
    main()

---------------- REPORT ----------------
Total income: $2608 

Current number of tools in store: 8 

Names of tools in store: T14-Concrete, T13-Plumbing, T01-Plumbing, T09-Concrete, T08-Yardwork, T02-Woodwork, T03-Yardwork, T05-Plumbing 

Rentals which are active: 
	1. Customer: C01-R
	Tools rented: T12-Woodwork
	Rental length: 5 days
	Cost: $20

	2. Customer: C01-R
	Tools rented: T11-Woodwork
	Rental length: 5 days
	Cost: $20

	3. Customer: C07-R
	Tools rented: T16-Concrete
	Rental length: 3 days
	Cost: $6

	4. Customer: C02-R
	Tools rented: T17-Yardwork, T06-Yardwork, T07-Yardwork
	Rental length: 5 days
	Cost: $75

	5. Customer: C04-B
	Tools rented: T19-Plumbing, T10-Yardwork, T18-Plumbing
	Rental length: 7 days
	Cost: $77

	6. Customer: C09-C
	Tools rented: T15-Concrete
	Rental length: 2 days
	Cost: $4

	7. Customer: C06-C
	Tools rented: T04-Yardwork, T20-Woodwork
	Rental length: 1 days
	Cost: $9


Rentals whcih completed: 
	1. Customer: C07-R
	Tools rented: T13-Plumbing
	Rental len